In [1]:
import pandas as pd
import psycopg2
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
conn = psycopg2.connect(dbname="adsdb", user="adsdb", password="adsdb")

In [3]:
df_final = pd.read_sql('SELECT * from exploitation.demographic_deaths;', conn, params = dict(name="adsdb"))

# Feature selection

In [4]:
df_final.shape

(2014, 37)

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split, RepeatedKFold, GridSearchCV
from sklearn.inspection import permutation_importance

In [6]:
df_final.columns

Index(['Year', 'Country', 'Deaths', 'Name',
       'Fertility rate, total (births per woman)', 'GDP (constant LCU)',
       'GDP (current LCU)', 'GDP (current US$)',
       'GDP deflator (base year varies by country)', 'GDP growth (annual %)',
       'GDP per capita (current LCU)', 'GDP per capita (current US$)',
       'GDP per capita, PPP (constant 2011 international $)',
       'GDP per capita, PPP (current international $)',
       'GDP, PPP (constant 2011 international $)',
       'GDP, PPP (current international $)', 'GNI (current LCU)',
       'GNI per capita (current LCU)',
       'GNI per capita, Atlas method (current US$)',
       'GNI per capita, PPP (current international $)',
       'General government total expenditure (current LCU)',
       'Life expectancy at birth, total (years)',
       'Mortality rate, infant (per 1,000 live births)',
       'Official exchange rate (LCU per US$, period average)',
       'PPP conversion factor, GDP (LCU per international $)',
       '

In [7]:
# Droping columns with NaN in predictor
df = df_final[df_final['Deaths'].notna()]
df = df[df['Total population (thousands)'].notna()]
df['Deaths/Pop'] = (df['Deaths']/(df['Total population (thousands)']*1000))*100
df = df.drop(columns=['Deaths', 'Name'])
df.head()

,Year,Country,"Fertility rate, total (births per woman)",GDP (constant LCU),GDP (current LCU),GDP (current US$),GDP deflator (base year varies by country),GDP growth (annual %),GDP per capita (current LCU),GDP per capita (current US$),...,Population aged 25-64 years (thousands),Population aged 65 years or older (thousands),Population growth (annual %),Poverty headcount ratio at $3.20 a day (PPP) (% of population),"Prevalence of HIV, total (% of population ages 15-49)",Price level ratio of PPP conversion factor (GDP) to market exch,Rural population (% of total population),Total debt service (% of GNI),Total population (thousands),Deaths/Pop
0,1994,3380,1.911,4.688180e+12,3.689090e+12,1.466835e+11,78.68917,7.99702,62659.41008,2491.42784,...,27588.383,3098.052,0.91714,19.2,1.9,0.39664,69.895,5.46978,58875.269,0.518938
1,1994,4050,1.810,1.480126e+12,9.932865e+11,1.561624e+11,67.10822,5.33246,190789.88375,29995.57962,...,2800.528,800.323,0.33771,NaN,NaN,1.38630,15.048,NaN,5211.334,1.172425
2,1994,4274,1.670,3.442415e+10,1.699534e+10,2.016294e+10,49.37039,6.20553,3178.87894,3771.35952,...,2646.994,571.808,0.39405,NaN,NaN,0.46834,43.402,NaN,5362.896,0.958120
3,1995,2320,NaN,NaN,1.858871e+08,NaN,NaN,NaN,18875.62134,NaN,...,5.183,0.488,NaN,NaN,NaN,NaN,NaN,NaN,9.848,1.015435
4,1995,2430,3.072,9.077419e+09,3.056830e+08,6.915905e+08,3.36751,0.00000,691.82371,1565.21202,...,186.966,22.568,1.53991,NaN,0.8,0.33136,33.924,NaN,441.851,0.347402


In [8]:
# Train test datasets
train=df.sample(frac=0.75,random_state=200)
test=df.drop(train.index)

y_train = train["Deaths/Pop"]
X_train = train.drop(columns ="Deaths/Pop")
y_test = test["Deaths/Pop"]
X_test = test.drop(columns ="Deaths/Pop")

X_train.shape, X_test.shape

((1365, 35), (455, 35))

In [9]:
X_train.to_csv('./datasets/train_dataset')

In [10]:
X_test.to_csv('./datasets/test_dataset')